In [ ]:
import sagemaker
import boto3
import numpy as np                                # For performing matrix operations and numerical processing
import pandas as pd                               # For manipulating tabular data
from time import gmtime, strftime
import os
import io
import re

import plotly.express as px

pd.set_option('display.max_columns', 500)

## Store Order

In [ ]:
conn = boto3.client('s3')
bucket='cdh-audi-in-s3-prod-reporting-myaudi-eu-w1'
subfolder='reporting/store-order/'
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

list_so=[]
for f in contents:
    list_so.append(f['Key'])

In [ ]:
myAudi_SO = pd.DataFrame()
for i in list_so:
    data_location = 's3://{}/{}'.format(bucket,i)
#     print(data_location)
    df1 = pd.read_parquet(data_location, engine='pyarrow')
    myAudi_SO=myAudi_SO.append(df1)

In [ ]:
myAudi_SO

In [ ]:
myAudi_SO.columns

## Order Line Item

In [ ]:
conn = boto3.client('s3')
bucket='cdh-audi-in-s3-prod-reporting-myaudi-eu-w1'
subfolder='reporting/order-line-items/'
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

list_so=[]
for f in contents:
    list_so.append(f['Key'])
    print(f['Key'])

In [ ]:
myAudi_oli = pd.DataFrame()
data_location = 's3://{}/{}'.format(bucket,list_so[0])
#     print(data_location)
df1 = pd.read_parquet(data_location, engine='pyarrow')
myAudi_oli=myAudi_oli.append(df1)

myAudi_oli

# Varient

In [ ]:
conn = boto3.client('s3')
bucket='cdh-audi-in-s3-prod-reporting-myaudi-eu-w1'
subfolder='reporting/variants/'
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

list_so=[]
for f in contents:
    list_so.append(f['Key'])

In [ ]:
myAudi_Varients = pd.DataFrame()
for i in list_so:
    data_location = 's3://{}/{}'.format(bucket,i)
#     print(data_location)
    df1 = pd.read_parquet(data_location, engine='pyarrow')
    myAudi_Varients=myAudi_Varients.append(df1)

In [ ]:
myAudi_Varients

In [ ]:
myAudi_Varients['id'].nunique()

## Merging SO and OLI

In [ ]:
df_merge_SO_OLI = pd.merge(myAudi_SO, myAudi_oli, left_on='id',right_on='orderId', how='inner')

In [ ]:
df_merge_SO_OLI

In [ ]:
df_merge_SO_OLI.columns

# Merge OLI, SO and Varient

In [ ]:
df_merge_SO_OLI_Varient = pd.merge(df_merge_SO_OLI, myAudi_Varients, left_on='itemVariantId',right_on='id', how='inner')

In [ ]:
df_merge_SO_OLI_Varient

In [ ]:
(df_merge_SO_OLI_Varient['title'].nunique())

In [ ]:
(df_merge_SO_OLI_Varient['name'].nunique())

In [ ]:
df_merge_SO_OLI['name'].nunique()

In [ ]:
df_merge_SO_OLI_Varient['title'].nunique()

In [ ]:
df_merge_SO_OLI_Varient.columns.tolist()

## Purchase Date

In [ ]:
df_merge_SO_OLI_Varient['year']=df_merge_SO_OLI_Varient['purchasedDate'].dt.year

In [ ]:
df_merge_SO_OLI_Varient['year'].value_counts()

In [ ]:
df_merge_SO_OLI_Varient['month']=df_merge_SO_OLI_Varient['purchasedDate'].dt.month

In [ ]:
df_merge_SO_OLI_Varient['month'].value_counts()

In [ ]:
df_merge_SO_OLI_Varient['purchasedDate'] = df_merge_SO_OLI_Varient['purchasedDate'].dt.date

In [ ]:
df_merge_SO_OLI_Varient['month-year']=pd.to_datetime(df_merge_SO_OLI_Varient['purchasedDate']).dt.strftime('%Y-%m')

In [ ]:
df_merge_SO_OLI_Varient.columns.tolist()

# Composite Key

In [ ]:
df_merge_SO_OLI_Varient['name_varientId']=df_merge_SO_OLI_Varient['name']+df_merge_SO_OLI_Varient['itemVariantId']

In [ ]:
df_merge_SO_OLI_Varient['name_varientId'].nunique()

In [ ]:
df_merge_SO_OLI_Varient

# Check Product available in varient table but not in oli

In [ ]:
df_merge_SO_OLI_Varient['name'].nunique()

In [ ]:
myAudi_Varients['id'].nunique()

In [ ]:
lst_Varient_Id = []
lst_oli_VarientId= []

In [ ]:
lst_Varient_Id = myAudi_Varients['id'].unique()

In [ ]:
lst_Varient_Id

In [ ]:
lst_oli_VarientId = df_merge_SO_OLI_Varient['itemVariantId'].unique()

In [ ]:
lst_oli_VarientId

In [ ]:
len(lst_oli_VarientId)

In [ ]:
df_merge_SO_OLI_Varient.columns.tolist()

# Preparing Data

## Dataframe with purchase date, name and count

In [ ]:
df_merge_product = df_merge_SO_OLI_Varient.groupby(['month-year','name_varientId'])['quantity_x'].sum().reset_index(name="count")

In [ ]:
df_merge_product

In [ ]:
pd.pivot_table(df_merge_product,index=['month-year'],columns=['name_varientId'])

In [ ]:
df_merge_product.groupby('month-year')['month-year'].count()

# HTS

In [ ]:
import hts

## Data

In [ ]:
df_total = df_merge_product.groupby('month-year')['count'].count().reset_index(name='total')

In [ ]:
df_total.set_index('month-year',inplace=True)

In [ ]:
df_total

In [ ]:
df_total.max()

In [ ]:
df_total.min()

In [ ]:
df_total.mean()

## Product Dataframe

In [ ]:
df_merge_product

In [ ]:
df_product = df_merge_product.groupby(['month-year','name_varientId'])['count'].sum().reset_index(name='count').pivot(index="month-year", columns="name_varientId", values="count")

In [ ]:
df_product

## Product Total

In [ ]:
# Product_total=df_product.join(df_total)
model_total=df_product.join(df_total)
model_total.index = pd.to_datetime(model_total.index)
model_total = model_total.resample("M") \
                           .sum()

In [ ]:
model_total

## Product Hierarchy

In [ ]:
Product_hierarchy = {'total': list(df_merge_product['name_varientId'].unique())}

In [ ]:
Product_hierarchy

## Creating Hierarchy

In [ ]:
from hts.hierarchy import HierarchyTree

ht = HierarchyTree.from_nodes(nodes=Product_hierarchy, df=model_total)
ht

## Visualization

In [ ]:
model_total["total"].plot(title="Product - total level");

In [ ]:
ax = model_total[Product_hierarchy['total']].plot(title="Product - state level")
ax.legend(bbox_to_anchor=(1.0, 1.0));

## Model

In [ ]:
model_total_test = model_total[-4:]

In [ ]:
model_total_test

In [ ]:
model_total_new = model_total[:-4]

## OLS — the optimal combination using OLS

In [ ]:
model_ols_arima = hts.HTSRegressor(model='auto_arima', revision_method='OLS', n_jobs=0)

In [ ]:
model_ols_arima = model_ols_arima.fit(model_total_new, Product_hierarchy)

In [ ]:
pred_ols_arima = model_ols_arima.predict(steps_ahead=4)

In [ ]:
pred_ols_arima

## Converting -ve Forecasted  values to +ve

In [ ]:
import pandas as pd


# create a copy of the original dataframe for the forecast
pred_ols_arima_forecast = pred_ols_arima.copy()

# convert negative values to positive in the forecast
pred_ols_arima_forecast[pred_ols_arima_forecast < 0] = -pred_ols_arima_forecast[pred_ols_arima_forecast < 0]

# combine the original and forecasted dataframes
df_combined = pd.concat([pred_ols_arima, pred_ols_arima_forecast], keys=['actual', 'forecast'])
df_comb=df_combined

# print the result
print(df_combined)

In [ ]:
df_pred_positive=pd.DataFrame(df_combined)

In [ ]:
df_pred_positive

In [ ]:
model_total_test

model_total_new['total'].plot(legend=True , figsize=(15,5))
df_pred_positive['total'][-4:].plot(legend=True)
model_total_test['total'].plot(legend=True)

## Stationality

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def ad_test(dataset):
    dftest = adfuller(dataset, autolag = 'AIC')
    print("1. ADF : ",dftest[0])
    print("2. P-Value : ", dftest[1])
    print("3. Num Of Lags : ", dftest[2])
    print("4. Num Of Observations Used For ADF Regression:",dftest[3])
    print("5. Critical Values :")
    for key, val in dftest[4].items():
        print("\t",key, ": ", val)
ad_test(model_total["total"])

## SMAPE

In [ ]:
#On test train split data (out sample)

In [ ]:
def smape(actual,forecast):
    return 100/len(actual) * np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast)))
Actual = np.array([125, 173, 146, 116])
Forecast = np.array([115.798700, 115.763451, 115.709052, 115.689475])
result = smape(Actual, Forecast)
print("SMAPE :", result)

###def mape(actual,forecast):
    return 1/len(actual) * np.sum(np.abs((actual - forecast)/actual))
output = 100
for i in Models:
    for j in Revision_methods:
        best_model = hts.HTSRegressor(model=i, revision_method=j, n_jobs=0)
        best_model = best_model.fit(model_total_new, Product_hierarchy)
        pred_best_model = best_model.predict(steps_ahead=4)
        lst_act = []
        lst_forcst = []
        for l in range(4):
            lst_act.append(np.array(model_total_test['total'][l]))
            lst_forcst.append(np.array(pred_best_model['total'][18+l]))
        lst_act = np.array(lst_act)
        lst_forcst = np.array(lst_forcst)
        mape = 100/len(lst_act) * np.sum(np.abs(lst_act - lst_forcst)/np.abs(lst_act))
        print(mape,i,j)
        print(lst_act,lst_forcst)
        if mape<output:
            output=mape
            model=i
            revesion_model=j
print(output,model,revesion_model)
###

# Revenue

In [ ]:
df_merge_SO_OLI_Varient

In [ ]:
df_pred_positive

In [ ]:
df_pred_positive_test=pd.DataFrame(df_pred_positive["total"].loc["forecast"][-4:])

In [ ]:
df_pred_positive_test

In [ ]:
df_pred_positive_test['total']=df_pred_positive_test['total'].astype(float)

In [ ]:
costing=df_merge_SO_OLI_Varient[["costPerItem", 'name_varientId']]

In [ ]:
costing["name_varientId"].nunique()

In [ ]:
costing

In [ ]:
costing[costing["costPerItem"]==200000.0]

In [ ]:
costing.reset_index(inplace=True)

In [ ]:
costing['name_varientId'].unique()

In [ ]:
df_pred_positive_test

In [ ]:
df_pred_positive

# Ceil

In [ ]:
df_pred_positive

In [ ]:
pred_ceil=df_pred_positive.copy()

In [ ]:
import math

In [ ]:
ceil_func = lambda x: math.ceil(x) if isinstance(x, (int, float)) else x

In [ ]:
pred_ceil.iloc[:, 1:] = pred_ceil.iloc[:, 1:].applymap(ceil_func)

In [ ]:
pred_ceil

In [ ]:
for i in costing['name_varientId'].unique():
    pred_ceil[i]=pred_ceil[i]*costing[costing['name_varientId']==i]['costPerItem'].values[0]

In [ ]:
pred_ceil

In [ ]:
pred_ceil2=pred_ceil[-4:]

In [ ]:
pred_ceil2

In [ ]:
pred_ceil2["total"]=0

In [ ]:
pred_ceil2['total'] = pred_ceil.apply(lambda row: row.sum(), axis=1)

In [ ]:
pred_ceil2

In [ ]:
pred_ceil2.drop("total",inplace=True,axis=1)

In [ ]:
pred_ceil2["total_floor"]=0

In [ ]:
pred_ceil2['total_ceil'] = pred_ceil2.apply(lambda row: row.sum(), axis=1)

In [ ]:
pred_ceil2.drop("total_floor",inplace=True,axis=1)

In [ ]:
pred_ceil2

# Floor

In [ ]:
pred_floor=df_pred_positive.copy()

In [ ]:
pred_floor

In [ ]:
floor_func = lambda x: math.ceil(x) if isinstance(x, (int, float)) else x

In [ ]:
pred_floor.iloc[:, 1:] = pred_floor.iloc[:, 1:].applymap(ceil_func)

In [ ]:
pred_floor

In [ ]:
for i in costing['name_varientId'].unique():
    pred_floor[i]=pred_floor[i]*costing[costing['name_varientId']==i]['costPerItem'].values[0]

In [ ]:
pred_floor

In [ ]:
pred_floor2=pred_floor[-4:]

In [ ]:
pred_floor2.drop("total",inplace=True,axis=1)

In [ ]:
pred_floor2["total_floor"]=0

In [ ]:
pred_floor2['total_floor'] = pred_floor2.apply(lambda row: row.sum(), axis=1)

In [ ]:
pred_floor2

In [ ]:
pred_ceil2["total_ceil"]

In [ ]:
pred_floor2["total_floor"]

In [ ]:
result_c= pd.DataFrame(pred_ceil2,columns=["total_ceil"])

In [ ]:
result_c

In [ ]:
result_f

In [ ]:
model_total_test

In [ ]:
costing

In [ ]:
for i in costing['name_varientId'].unique():
    model_total_test[i]=model_total_test[i]*costing[costing['name_varientId']==i]['costPerItem'].values[0]

In [ ]:
model_total_test

In [ ]:
model_total_test["total_original"]=0

In [ ]:
model_total_test['total_original'] = model_total_test.apply(lambda row: row.sum(), axis=1)

In [ ]:
model_total_test

In [ ]:
model_total_test["total_original"]

In [ ]:
result_org= pd.DataFrame(model_total_test,columns=["total_original"])

In [1438]:
result_org

,total_original
month-year,
2022-11-30,49278478.0
2022-12-31,61794569.0
2023-01-31,53618304.0
2023-02-28,52069893.0


In [1439]:
result_c

total_ceil
forecast 2022-11-30  57531495.0
         2022-12-31  55539491.0
         2023-01-31  51707214.0
         2023-02-28  50298604.0

In [1440]:
result_f

total_floor
forecast 2022-11-30   57290392.0
         2022-12-31   55295954.0
         2023-01-31   51466111.0
         2023-02-28   50056818.0

In [1441]:
result_org

,total_original
month-year,
2022-11-30,49278478.0
2022-12-31,61794569.0
2023-01-31,53618304.0
2023-02-28,52069893.0


## converting -ve values to 0

In [ ]:
pred_ols_arima

In [ ]:
import pandas as pd


# create a copy of the original dataframe for the forecast
pred_ols_arima_forecast = pred_ols_arima.copy()

# convert negative values to positive in the forecast
#pred_ols_arima_forecast[pred_ols_arima_forecast < 0] = -pred_ols_arima_forecast[pred_ols_arima_forecast < 0]
pred_ols_arima_forecast[pred_ols_arima_forecast < 0] = 0

# combine the original and forecasted dataframes
df_combined2 = pd.concat([pred_ols_arima, pred_ols_arima_forecast], keys=['actual', 'forecast'])
df_comb=df_combined2

# print the result
print(df_combined2)

In [ ]:
df_pred_zero=pd.DataFrame(df_combined2.loc["forecast"])

In [ ]:
df_pred_zero

# Ceil_zero

In [1461]:
pred_ceil_zero=df_pred_zero.copy()

In [1462]:
ceil_func_zero = lambda x: math.ceil(x) if isinstance(x, (int, float)) else x

In [1463]:
pred_ceil_zero.iloc[:, 1:] = pred_ceil_zero.iloc[:, 1:].applymap(ceil_func_zero)

In [1464]:
for i in costing['name_varientId'].unique():
    pred_ceil_zero[i]=pred_ceil_zero[i]*costing[costing['name_varientId']==i]['costPerItem'].values[0]

In [1465]:
pred_ceil_zero

,total,2-in-1 Display Cleaner278,Audi A4 Luggage Compartment Shell407,"Audi A5 Convertible, Floret Silver, 1:43427","Audi Aviator Sunglasses, Gun Metal428","Audi Baseball Cap, Black117","Audi Baseball Cap, Blue207","Audi Baseball Cap, Red121","Audi Q5, Navarra Blue, 1:43426","Audi Rs 5 Coupe, 1:43181","Audi Smart Urban Travelbag, Grey157","Audi Sport Baseball cap, Kids79","Audi Sport Mug Carbon, Grey71","Audi Sport Number plate holder, Black119","Audi USB Memory Key, 8 GB281",Battery Charger381,Business Bag373,Cockpit Care Solution406,LED gooseneck reading lamp292,Leather Care Set285,Pocket Umbrella130,"Ready-mixed Windscreen washer fluid, 3 l273",Rear Seat Bag382,Seat Back Pocket375,Size L143,Size: 158/164152,Valve Caps287,"Wheel Cleaner Gel, 500 ml276",Wheelsens288,Women's Watch72,"heritage Auto Union Streamline, Silver 1:43170",myAudi Connect80,"quattro Mug, White/black123","quattro Umbrella, Black84",55 TFSI quattro tiptronic113,55 TFSI quattro tiptronic114,Audi A6 45 TFSI Premium Plus108,Audi A6 45 TFSI Technology107,"Audi A6 Car Cover, Audi Rings399","Audi A7 Sportback, Myth Black, 1:43183",Audi Baby Mirror279,"Audi Backpack Foldable, Black201","Audi Child Seat, misano red/black402","Audi Cufflinks, Black75","Audi Q2 All-weather Floor Mats, Front301","Audi Q2 All-weather Floor Mats, Rear289","Audi Q8, Orca Black, 1:43126","Audi R8 Coupé, Camouflage Green Matt, 1:18420","Audi R8 Lms Presentation Warpaint, 1:43164",Audi RS 7 Sportback tiptronic112,"Audi Rings Decals, Floret Silver298","Audi S5 Convertible, 1:43176","Audi Smart Urban Backpack, Grey142",Audi Sport In Ear plugs60,"Audi Sport Sunglasses mirror lens, Black/red66","Audi Sport Umbrella, Black/red, Big133","Audi Umbrella, Black/silver, Big132",Espresso Mobile291,"Insect Eliminator, 500 ml274",Loading Sill Protective Mat307,Microfibre Interior Brush280,"Notebook, Din A5144",Premium plus111,Premium99,RS7 Sportback tiptronic112,RSQ8 4.0 TFSI quattro116,S5 TFSI109,Size L135,Stainless Steel Pedal Caps356,Standard with sunroof100,TEST1425,Technology104,Technology110,Tow Rope282,USB Charging Adapter (dual)297,Wireless Charging Cover for Apple iPhone 6/6S325,Wireless Charging Pad320,"heritage Sunglasses, Black/red156","quattro Ballpoint Pen With USB, 8 GB136","quattro Key Ring Carabiner, Black/silver124","Audi A4 Wind Deflector, Front364","Audi A8 L, Monsoon Grey, 1:43122","Audi Baseball Cap, White202","Audi Q7 Rubber Floor Mats, Front327","Audi Smartphone Case, Dark grey, iPhone 6/6s/7/8433","Audi Sport Mug, Red141","Audi e-tron, Antigua Blue, 1:18429",Celebration 55 TFSI quattro tiptronic115,Coat Hanger290,"Cool and Hot Box, 20 l408",Heritage wallet76,"Intensive plastic care solution, 250 ml268","Leather Care Solution, 250 ml270",Microfibre Polishing cloth267,"Plastic Cleaner, 500 ml269","Roof Luggage Box, 300 l414","Roof Luggage Box, 360 l415","Screenwash, 500 ml264",Size L175,Size L199,Size L216,Size M165,Size S184,Size S214,Size S442,Size XL160,TEST2447,"USB type-C Charging Cable Set, for Lightning & Type C devices355",e-tron Booking449,e-tron Sportback Booking451,"heritage Auto Union Typ B, Silver 1:43171","heritage Spark plug USB, 8 GB204",quattro Logo in Black339,quattro Messenger Bag203,"Active Foam Cleaner, 300 ml272",Adapter Cable Set for the Audi Music Interface477,"All-weather Floor Mats, Rear481",Anti-Marten Device393,"Audi A3 All-Weather Floor Mats, Rear491","Audi A3 Rubber Floor Mats, Rear500","Audi A6 All-weather Floor Mats, Front479",Audi A6 Bootliner380,"Audi Baby Seat, Misano red/black419",Audi Ballpoint pen205,"Audi Card Case, Black127","Audi Entertainment Mobile, Twin Package Click & Go505",Audi Q3 Bootliner494,"Audi Q3 Rubber Floor Mats, Rear466","Audi Q3 Trim Strips for Radiator Grille, Chrome Finish474",Audi Q5 Luggage Compartment Shell533,"Audi Q5 Rubber Floor Mats, Rear497",Audi Q7 Stainless Steel Pedal Caps454,"Audi Q8 All-weather Floor Mats, Rear318",Audi Q8 Bootliner383,"Audi S8 limited, Navarra Blue, 1:43150",Audi Sport Ballpointpen197,"Audi

In [1466]:
pred_ceil2_zero=pred_ceil_zero[-4:]

In [1467]:
pred_ceil2_zero

,total,2-in-1 Display Cleaner278,Audi A4 Luggage Compartment Shell407,"Audi A5 Convertible, Floret Silver, 1:43427","Audi Aviator Sunglasses, Gun Metal428","Audi Baseball Cap, Black117","Audi Baseball Cap, Blue207","Audi Baseball Cap, Red121","Audi Q5, Navarra Blue, 1:43426","Audi Rs 5 Coupe, 1:43181","Audi Smart Urban Travelbag, Grey157","Audi Sport Baseball cap, Kids79","Audi Sport Mug Carbon, Grey71","Audi Sport Number plate holder, Black119","Audi USB Memory Key, 8 GB281",Battery Charger381,Business Bag373,Cockpit Care Solution406,LED gooseneck reading lamp292,Leather Care Set285,Pocket Umbrella130,"Ready-mixed Windscreen washer fluid, 3 l273",Rear Seat Bag382,Seat Back Pocket375,Size L143,Size: 158/164152,Valve Caps287,"Wheel Cleaner Gel, 500 ml276",Wheelsens288,Women's Watch72,"heritage Auto Union Streamline, Silver 1:43170",myAudi Connect80,"quattro Mug, White/black123","quattro Umbrella, Black84",55 TFSI quattro tiptronic113,55 TFSI quattro tiptronic114,Audi A6 45 TFSI Premium Plus108,Audi A6 45 TFSI Technology107,"Audi A6 Car Cover, Audi Rings399","Audi A7 Sportback, Myth Black, 1:43183",Audi Baby Mirror279,"Audi Backpack Foldable, Black201","Audi Child Seat, misano red/black402","Audi Cufflinks, Black75","Audi Q2 All-weather Floor Mats, Front301","Audi Q2 All-weather Floor Mats, Rear289","Audi Q8, Orca Black, 1:43126","Audi R8 Coupé, Camouflage Green Matt, 1:18420","Audi R8 Lms Presentation Warpaint, 1:43164",Audi RS 7 Sportback tiptronic112,"Audi Rings Decals, Floret Silver298","Audi S5 Convertible, 1:43176","Audi Smart Urban Backpack, Grey142",Audi Sport In Ear plugs60,"Audi Sport Sunglasses mirror lens, Black/red66","Audi Sport Umbrella, Black/red, Big133","Audi Umbrella, Black/silver, Big132",Espresso Mobile291,"Insect Eliminator, 500 ml274",Loading Sill Protective Mat307,Microfibre Interior Brush280,"Notebook, Din A5144",Premium plus111,Premium99,RS7 Sportback tiptronic112,RSQ8 4.0 TFSI quattro116,S5 TFSI109,Size L135,Stainless Steel Pedal Caps356,Standard with sunroof100,TEST1425,Technology104,Technology110,Tow Rope282,USB Charging Adapter (dual)297,Wireless Charging Cover for Apple iPhone 6/6S325,Wireless Charging Pad320,"heritage Sunglasses, Black/red156","quattro Ballpoint Pen With USB, 8 GB136","quattro Key Ring Carabiner, Black/silver124","Audi A4 Wind Deflector, Front364","Audi A8 L, Monsoon Grey, 1:43122","Audi Baseball Cap, White202","Audi Q7 Rubber Floor Mats, Front327","Audi Smartphone Case, Dark grey, iPhone 6/6s/7/8433","Audi Sport Mug, Red141","Audi e-tron, Antigua Blue, 1:18429",Celebration 55 TFSI quattro tiptronic115,Coat Hanger290,"Cool and Hot Box, 20 l408",Heritage wallet76,"Intensive plastic care solution, 250 ml268","Leather Care Solution, 250 ml270",Microfibre Polishing cloth267,"Plastic Cleaner, 500 ml269","Roof Luggage Box, 300 l414","Roof Luggage Box, 360 l415","Screenwash, 500 ml264",Size L175,Size L199,Size L216,Size M165,Size S184,Size S214,Size S442,Size XL160,TEST2447,"USB type-C Charging Cable Set, for Lightning & Type C devices355",e-tron Booking449,e-tron Sportback Booking451,"heritage Auto Union Typ B, Silver 1:43171","heritage Spark plug USB, 8 GB204",quattro Logo in Black339,quattro Messenger Bag203,"Active Foam Cleaner, 300 ml272",Adapter Cable Set for the Audi Music Interface477,"All-weather Floor Mats, Rear481",Anti-Marten Device393,"Audi A3 All-Weather Floor Mats, Rear491","Audi A3 Rubber Floor Mats, Rear500","Audi A6 All-weather Floor Mats, Front479",Audi A6 Bootliner380,"Audi Baby Seat, Misano red/black419",Audi Ballpoint pen205,"Audi Card Case, Black127","Audi Entertainment Mobile, Twin Package Click & Go505",Audi Q3 Bootliner494,"Audi Q3 Rubber Floor Mats, Rear466","Audi Q3 Trim Strips for Radiator Grille, Chrome Finish474",Audi Q5 Luggage Compartment Shell533,"Audi Q5 Rubber Floor Mats, Rear497",Audi Q7 Stainless Steel Pedal Caps454,"Audi Q8 All-weather Floor Mats, Rear318",Audi Q8 Bootliner383,"Audi S8 limited, Navarra Blue, 1:43150",Audi Sport Ballpointpen197,"Audi

In [1468]:
pred_ceil2_zero.drop("total",inplace=True,axis=1)

/tmp/ipykernel_31827/3712534053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_ceil2_zero.drop("total",inplace=True,axis=1)


In [1469]:
pred_ceil2_zero['total_ceil_zero'] = pred_ceil2_zero.apply(lambda row: row.sum(), axis=1)

/tmp/ipykernel_31827/662332970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_ceil2_zero['total_ceil_zero'] = pred_ceil2_zero.apply(lambda row: row.sum(), axis=1)


In [1470]:
pred_ceil2_zero

,2-in-1 Display Cleaner278,Audi A4 Luggage Compartment Shell407,"Audi A5 Convertible, Floret Silver, 1:43427","Audi Aviator Sunglasses, Gun Metal428","Audi Baseball Cap, Black117","Audi Baseball Cap, Blue207","Audi Baseball Cap, Red121","Audi Q5, Navarra Blue, 1:43426","Audi Rs 5 Coupe, 1:43181","Audi Smart Urban Travelbag, Grey157","Audi Sport Baseball cap, Kids79","Audi Sport Mug Carbon, Grey71","Audi Sport Number plate holder, Black119","Audi USB Memory Key, 8 GB281",Battery Charger381,Business Bag373,Cockpit Care Solution406,LED gooseneck reading lamp292,Leather Care Set285,Pocket Umbrella130,"Ready-mixed Windscreen washer fluid, 3 l273",Rear Seat Bag382,Seat Back Pocket375,Size L143,Size: 158/164152,Valve Caps287,"Wheel Cleaner Gel, 500 ml276",Wheelsens288,Women's Watch72,"heritage Auto Union Streamline, Silver 1:43170",myAudi Connect80,"quattro Mug, White/black123","quattro Umbrella, Black84",55 TFSI quattro tiptronic113,55 TFSI quattro tiptronic114,Audi A6 45 TFSI Premium Plus108,Audi A6 45 TFSI Technology107,"Audi A6 Car Cover, Audi Rings399","Audi A7 Sportback, Myth Black, 1:43183",Audi Baby Mirror279,"Audi Backpack Foldable, Black201","Audi Child Seat, misano red/black402","Audi Cufflinks, Black75","Audi Q2 All-weather Floor Mats, Front301","Audi Q2 All-weather Floor Mats, Rear289","Audi Q8, Orca Black, 1:43126","Audi R8 Coupé, Camouflage Green Matt, 1:18420","Audi R8 Lms Presentation Warpaint, 1:43164",Audi RS 7 Sportback tiptronic112,"Audi Rings Decals, Floret Silver298","Audi S5 Convertible, 1:43176","Audi Smart Urban Backpack, Grey142",Audi Sport In Ear plugs60,"Audi Sport Sunglasses mirror lens, Black/red66","Audi Sport Umbrella, Black/red, Big133","Audi Umbrella, Black/silver, Big132",Espresso Mobile291,"Insect Eliminator, 500 ml274",Loading Sill Protective Mat307,Microfibre Interior Brush280,"Notebook, Din A5144",Premium plus111,Premium99,RS7 Sportback tiptronic112,RSQ8 4.0 TFSI quattro116,S5 TFSI109,Size L135,Stainless Steel Pedal Caps356,Standard with sunroof100,TEST1425,Technology104,Technology110,Tow Rope282,USB Charging Adapter (dual)297,Wireless Charging Cover for Apple iPhone 6/6S325,Wireless Charging Pad320,"heritage Sunglasses, Black/red156","quattro Ballpoint Pen With USB, 8 GB136","quattro Key Ring Carabiner, Black/silver124","Audi A4 Wind Deflector, Front364","Audi A8 L, Monsoon Grey, 1:43122","Audi Baseball Cap, White202","Audi Q7 Rubber Floor Mats, Front327","Audi Smartphone Case, Dark grey, iPhone 6/6s/7/8433","Audi Sport Mug, Red141","Audi e-tron, Antigua Blue, 1:18429",Celebration 55 TFSI quattro tiptronic115,Coat Hanger290,"Cool and Hot Box, 20 l408",Heritage wallet76,"Intensive plastic care solution, 250 ml268","Leather Care Solution, 250 ml270",Microfibre Polishing cloth267,"Plastic Cleaner, 500 ml269","Roof Luggage Box, 300 l414","Roof Luggage Box, 360 l415","Screenwash, 500 ml264",Size L175,Size L199,Size L216,Size M165,Size S184,Size S214,Size S442,Size XL160,TEST2447,"USB type-C Charging Cable Set, for Lightning & Type C devices355",e-tron Booking449,e-tron Sportback Booking451,"heritage Auto Union Typ B, Silver 1:43171","heritage Spark plug USB, 8 GB204",quattro Logo in Black339,quattro Messenger Bag203,"Active Foam Cleaner, 300 ml272",Adapter Cable Set for the Audi Music Interface477,"All-weather Floor Mats, Rear481",Anti-Marten Device393,"Audi A3 All-Weather Floor Mats, Rear491","Audi A3 Rubber Floor Mats, Rear500","Audi A6 All-weather Floor Mats, Front479",Audi A6 Bootliner380,"Audi Baby Seat, Misano red/black419",Audi Ballpoint pen205,"Audi Card Case, Black127","Audi Entertainment Mobile, Twin Package Click & Go505",Audi Q3 Bootliner494,"Audi Q3 Rubber Floor Mats, Rear466","Audi Q3 Trim Strips for Radiator Grille, Chrome Finish474",Audi Q5 Luggage Compartment Shell533,"Audi Q5 Rubber Floor Mats, Rear497",Audi Q7 Stainless Steel Pedal Caps454,"Audi Q8 All-weather Floor Mats, Rear318",Audi Q8 Bootliner383,"Audi S8 limited, Navarra Blue, 1:43150",Audi Sport Ballpointpen197,"Audi Tie, 

In [1471]:
pred_ceil2_zero["total_ceil_zero"]

2022-11-30    42481650.0
2022-12-31    40935877.0
2023-01-31    36920334.0
2023-02-28    35512600.0
Freq: M, Name: total_ceil_zero, dtype: float64

## floor_zero

In [1472]:
pred_floor_zero=df_pred_zero.copy()

In [1473]:
floor_func_zero = lambda x: math.floor(x) if isinstance(x, (int, float)) else x

In [1474]:
pred_floor_zero.iloc[:, 1:] = pred_floor_zero.iloc[:, 1:].applymap(floor_func_zero)

In [1475]:
for i in costing['name_varientId'].unique():
    pred_floor_zero[i]=pred_floor_zero[i]*costing[costing['name_varientId']==i]['costPerItem'].values[0]

In [1476]:
pred_floor_zero

,total,2-in-1 Display Cleaner278,Audi A4 Luggage Compartment Shell407,"Audi A5 Convertible, Floret Silver, 1:43427","Audi Aviator Sunglasses, Gun Metal428","Audi Baseball Cap, Black117","Audi Baseball Cap, Blue207","Audi Baseball Cap, Red121","Audi Q5, Navarra Blue, 1:43426","Audi Rs 5 Coupe, 1:43181","Audi Smart Urban Travelbag, Grey157","Audi Sport Baseball cap, Kids79","Audi Sport Mug Carbon, Grey71","Audi Sport Number plate holder, Black119","Audi USB Memory Key, 8 GB281",Battery Charger381,Business Bag373,Cockpit Care Solution406,LED gooseneck reading lamp292,Leather Care Set285,Pocket Umbrella130,"Ready-mixed Windscreen washer fluid, 3 l273",Rear Seat Bag382,Seat Back Pocket375,Size L143,Size: 158/164152,Valve Caps287,"Wheel Cleaner Gel, 500 ml276",Wheelsens288,Women's Watch72,"heritage Auto Union Streamline, Silver 1:43170",myAudi Connect80,"quattro Mug, White/black123","quattro Umbrella, Black84",55 TFSI quattro tiptronic113,55 TFSI quattro tiptronic114,Audi A6 45 TFSI Premium Plus108,Audi A6 45 TFSI Technology107,"Audi A6 Car Cover, Audi Rings399","Audi A7 Sportback, Myth Black, 1:43183",Audi Baby Mirror279,"Audi Backpack Foldable, Black201","Audi Child Seat, misano red/black402","Audi Cufflinks, Black75","Audi Q2 All-weather Floor Mats, Front301","Audi Q2 All-weather Floor Mats, Rear289","Audi Q8, Orca Black, 1:43126","Audi R8 Coupé, Camouflage Green Matt, 1:18420","Audi R8 Lms Presentation Warpaint, 1:43164",Audi RS 7 Sportback tiptronic112,"Audi Rings Decals, Floret Silver298","Audi S5 Convertible, 1:43176","Audi Smart Urban Backpack, Grey142",Audi Sport In Ear plugs60,"Audi Sport Sunglasses mirror lens, Black/red66","Audi Sport Umbrella, Black/red, Big133","Audi Umbrella, Black/silver, Big132",Espresso Mobile291,"Insect Eliminator, 500 ml274",Loading Sill Protective Mat307,Microfibre Interior Brush280,"Notebook, Din A5144",Premium plus111,Premium99,RS7 Sportback tiptronic112,RSQ8 4.0 TFSI quattro116,S5 TFSI109,Size L135,Stainless Steel Pedal Caps356,Standard with sunroof100,TEST1425,Technology104,Technology110,Tow Rope282,USB Charging Adapter (dual)297,Wireless Charging Cover for Apple iPhone 6/6S325,Wireless Charging Pad320,"heritage Sunglasses, Black/red156","quattro Ballpoint Pen With USB, 8 GB136","quattro Key Ring Carabiner, Black/silver124","Audi A4 Wind Deflector, Front364","Audi A8 L, Monsoon Grey, 1:43122","Audi Baseball Cap, White202","Audi Q7 Rubber Floor Mats, Front327","Audi Smartphone Case, Dark grey, iPhone 6/6s/7/8433","Audi Sport Mug, Red141","Audi e-tron, Antigua Blue, 1:18429",Celebration 55 TFSI quattro tiptronic115,Coat Hanger290,"Cool and Hot Box, 20 l408",Heritage wallet76,"Intensive plastic care solution, 250 ml268","Leather Care Solution, 250 ml270",Microfibre Polishing cloth267,"Plastic Cleaner, 500 ml269","Roof Luggage Box, 300 l414","Roof Luggage Box, 360 l415","Screenwash, 500 ml264",Size L175,Size L199,Size L216,Size M165,Size S184,Size S214,Size S442,Size XL160,TEST2447,"USB type-C Charging Cable Set, for Lightning & Type C devices355",e-tron Booking449,e-tron Sportback Booking451,"heritage Auto Union Typ B, Silver 1:43171","heritage Spark plug USB, 8 GB204",quattro Logo in Black339,quattro Messenger Bag203,"Active Foam Cleaner, 300 ml272",Adapter Cable Set for the Audi Music Interface477,"All-weather Floor Mats, Rear481",Anti-Marten Device393,"Audi A3 All-Weather Floor Mats, Rear491","Audi A3 Rubber Floor Mats, Rear500","Audi A6 All-weather Floor Mats, Front479",Audi A6 Bootliner380,"Audi Baby Seat, Misano red/black419",Audi Ballpoint pen205,"Audi Card Case, Black127","Audi Entertainment Mobile, Twin Package Click & Go505",Audi Q3 Bootliner494,"Audi Q3 Rubber Floor Mats, Rear466","Audi Q3 Trim Strips for Radiator Grille, Chrome Finish474",Audi Q5 Luggage Compartment Shell533,"Audi Q5 Rubber Floor Mats, Rear497",Audi Q7 Stainless Steel Pedal Caps454,"Audi Q8 All-weather Floor Mats, Rear318",Audi Q8 Bootliner383,"Audi S8 limited, Navarra Blue, 1:43150",Audi Sport Ballpointpen197,"Audi

In [1477]:
pred_floor_zero

,total,2-in-1 Display Cleaner278,Audi A4 Luggage Compartment Shell407,"Audi A5 Convertible, Floret Silver, 1:43427","Audi Aviator Sunglasses, Gun Metal428","Audi Baseball Cap, Black117","Audi Baseball Cap, Blue207","Audi Baseball Cap, Red121","Audi Q5, Navarra Blue, 1:43426","Audi Rs 5 Coupe, 1:43181","Audi Smart Urban Travelbag, Grey157","Audi Sport Baseball cap, Kids79","Audi Sport Mug Carbon, Grey71","Audi Sport Number plate holder, Black119","Audi USB Memory Key, 8 GB281",Battery Charger381,Business Bag373,Cockpit Care Solution406,LED gooseneck reading lamp292,Leather Care Set285,Pocket Umbrella130,"Ready-mixed Windscreen washer fluid, 3 l273",Rear Seat Bag382,Seat Back Pocket375,Size L143,Size: 158/164152,Valve Caps287,"Wheel Cleaner Gel, 500 ml276",Wheelsens288,Women's Watch72,"heritage Auto Union Streamline, Silver 1:43170",myAudi Connect80,"quattro Mug, White/black123","quattro Umbrella, Black84",55 TFSI quattro tiptronic113,55 TFSI quattro tiptronic114,Audi A6 45 TFSI Premium Plus108,Audi A6 45 TFSI Technology107,"Audi A6 Car Cover, Audi Rings399","Audi A7 Sportback, Myth Black, 1:43183",Audi Baby Mirror279,"Audi Backpack Foldable, Black201","Audi Child Seat, misano red/black402","Audi Cufflinks, Black75","Audi Q2 All-weather Floor Mats, Front301","Audi Q2 All-weather Floor Mats, Rear289","Audi Q8, Orca Black, 1:43126","Audi R8 Coupé, Camouflage Green Matt, 1:18420","Audi R8 Lms Presentation Warpaint, 1:43164",Audi RS 7 Sportback tiptronic112,"Audi Rings Decals, Floret Silver298","Audi S5 Convertible, 1:43176","Audi Smart Urban Backpack, Grey142",Audi Sport In Ear plugs60,"Audi Sport Sunglasses mirror lens, Black/red66","Audi Sport Umbrella, Black/red, Big133","Audi Umbrella, Black/silver, Big132",Espresso Mobile291,"Insect Eliminator, 500 ml274",Loading Sill Protective Mat307,Microfibre Interior Brush280,"Notebook, Din A5144",Premium plus111,Premium99,RS7 Sportback tiptronic112,RSQ8 4.0 TFSI quattro116,S5 TFSI109,Size L135,Stainless Steel Pedal Caps356,Standard with sunroof100,TEST1425,Technology104,Technology110,Tow Rope282,USB Charging Adapter (dual)297,Wireless Charging Cover for Apple iPhone 6/6S325,Wireless Charging Pad320,"heritage Sunglasses, Black/red156","quattro Ballpoint Pen With USB, 8 GB136","quattro Key Ring Carabiner, Black/silver124","Audi A4 Wind Deflector, Front364","Audi A8 L, Monsoon Grey, 1:43122","Audi Baseball Cap, White202","Audi Q7 Rubber Floor Mats, Front327","Audi Smartphone Case, Dark grey, iPhone 6/6s/7/8433","Audi Sport Mug, Red141","Audi e-tron, Antigua Blue, 1:18429",Celebration 55 TFSI quattro tiptronic115,Coat Hanger290,"Cool and Hot Box, 20 l408",Heritage wallet76,"Intensive plastic care solution, 250 ml268","Leather Care Solution, 250 ml270",Microfibre Polishing cloth267,"Plastic Cleaner, 500 ml269","Roof Luggage Box, 300 l414","Roof Luggage Box, 360 l415","Screenwash, 500 ml264",Size L175,Size L199,Size L216,Size M165,Size S184,Size S214,Size S442,Size XL160,TEST2447,"USB type-C Charging Cable Set, for Lightning & Type C devices355",e-tron Booking449,e-tron Sportback Booking451,"heritage Auto Union Typ B, Silver 1:43171","heritage Spark plug USB, 8 GB204",quattro Logo in Black339,quattro Messenger Bag203,"Active Foam Cleaner, 300 ml272",Adapter Cable Set for the Audi Music Interface477,"All-weather Floor Mats, Rear481",Anti-Marten Device393,"Audi A3 All-Weather Floor Mats, Rear491","Audi A3 Rubber Floor Mats, Rear500","Audi A6 All-weather Floor Mats, Front479",Audi A6 Bootliner380,"Audi Baby Seat, Misano red/black419",Audi Ballpoint pen205,"Audi Card Case, Black127","Audi Entertainment Mobile, Twin Package Click & Go505",Audi Q3 Bootliner494,"Audi Q3 Rubber Floor Mats, Rear466","Audi Q3 Trim Strips for Radiator Grille, Chrome Finish474",Audi Q5 Luggage Compartment Shell533,"Audi Q5 Rubber Floor Mats, Rear497",Audi Q7 Stainless Steel Pedal Caps454,"Audi Q8 All-weather Floor Mats, Rear318",Audi Q8 Bootliner383,"Audi S8 limited, Navarra Blue, 1:43150",Audi Sport Ballpointpen197,"Audi

In [1478]:
pred_floor2_zero=pred_floor_zero[-4:]

In [1479]:
pred_floor2_zero

,total,2-in-1 Display Cleaner278,Audi A4 Luggage Compartment Shell407,"Audi A5 Convertible, Floret Silver, 1:43427","Audi Aviator Sunglasses, Gun Metal428","Audi Baseball Cap, Black117","Audi Baseball Cap, Blue207","Audi Baseball Cap, Red121","Audi Q5, Navarra Blue, 1:43426","Audi Rs 5 Coupe, 1:43181","Audi Smart Urban Travelbag, Grey157","Audi Sport Baseball cap, Kids79","Audi Sport Mug Carbon, Grey71","Audi Sport Number plate holder, Black119","Audi USB Memory Key, 8 GB281",Battery Charger381,Business Bag373,Cockpit Care Solution406,LED gooseneck reading lamp292,Leather Care Set285,Pocket Umbrella130,"Ready-mixed Windscreen washer fluid, 3 l273",Rear Seat Bag382,Seat Back Pocket375,Size L143,Size: 158/164152,Valve Caps287,"Wheel Cleaner Gel, 500 ml276",Wheelsens288,Women's Watch72,"heritage Auto Union Streamline, Silver 1:43170",myAudi Connect80,"quattro Mug, White/black123","quattro Umbrella, Black84",55 TFSI quattro tiptronic113,55 TFSI quattro tiptronic114,Audi A6 45 TFSI Premium Plus108,Audi A6 45 TFSI Technology107,"Audi A6 Car Cover, Audi Rings399","Audi A7 Sportback, Myth Black, 1:43183",Audi Baby Mirror279,"Audi Backpack Foldable, Black201","Audi Child Seat, misano red/black402","Audi Cufflinks, Black75","Audi Q2 All-weather Floor Mats, Front301","Audi Q2 All-weather Floor Mats, Rear289","Audi Q8, Orca Black, 1:43126","Audi R8 Coupé, Camouflage Green Matt, 1:18420","Audi R8 Lms Presentation Warpaint, 1:43164",Audi RS 7 Sportback tiptronic112,"Audi Rings Decals, Floret Silver298","Audi S5 Convertible, 1:43176","Audi Smart Urban Backpack, Grey142",Audi Sport In Ear plugs60,"Audi Sport Sunglasses mirror lens, Black/red66","Audi Sport Umbrella, Black/red, Big133","Audi Umbrella, Black/silver, Big132",Espresso Mobile291,"Insect Eliminator, 500 ml274",Loading Sill Protective Mat307,Microfibre Interior Brush280,"Notebook, Din A5144",Premium plus111,Premium99,RS7 Sportback tiptronic112,RSQ8 4.0 TFSI quattro116,S5 TFSI109,Size L135,Stainless Steel Pedal Caps356,Standard with sunroof100,TEST1425,Technology104,Technology110,Tow Rope282,USB Charging Adapter (dual)297,Wireless Charging Cover for Apple iPhone 6/6S325,Wireless Charging Pad320,"heritage Sunglasses, Black/red156","quattro Ballpoint Pen With USB, 8 GB136","quattro Key Ring Carabiner, Black/silver124","Audi A4 Wind Deflector, Front364","Audi A8 L, Monsoon Grey, 1:43122","Audi Baseball Cap, White202","Audi Q7 Rubber Floor Mats, Front327","Audi Smartphone Case, Dark grey, iPhone 6/6s/7/8433","Audi Sport Mug, Red141","Audi e-tron, Antigua Blue, 1:18429",Celebration 55 TFSI quattro tiptronic115,Coat Hanger290,"Cool and Hot Box, 20 l408",Heritage wallet76,"Intensive plastic care solution, 250 ml268","Leather Care Solution, 250 ml270",Microfibre Polishing cloth267,"Plastic Cleaner, 500 ml269","Roof Luggage Box, 300 l414","Roof Luggage Box, 360 l415","Screenwash, 500 ml264",Size L175,Size L199,Size L216,Size M165,Size S184,Size S214,Size S442,Size XL160,TEST2447,"USB type-C Charging Cable Set, for Lightning & Type C devices355",e-tron Booking449,e-tron Sportback Booking451,"heritage Auto Union Typ B, Silver 1:43171","heritage Spark plug USB, 8 GB204",quattro Logo in Black339,quattro Messenger Bag203,"Active Foam Cleaner, 300 ml272",Adapter Cable Set for the Audi Music Interface477,"All-weather Floor Mats, Rear481",Anti-Marten Device393,"Audi A3 All-Weather Floor Mats, Rear491","Audi A3 Rubber Floor Mats, Rear500","Audi A6 All-weather Floor Mats, Front479",Audi A6 Bootliner380,"Audi Baby Seat, Misano red/black419",Audi Ballpoint pen205,"Audi Card Case, Black127","Audi Entertainment Mobile, Twin Package Click & Go505",Audi Q3 Bootliner494,"Audi Q3 Rubber Floor Mats, Rear466","Audi Q3 Trim Strips for Radiator Grille, Chrome Finish474",Audi Q5 Luggage Compartment Shell533,"Audi Q5 Rubber Floor Mats, Rear497",Audi Q7 Stainless Steel Pedal Caps454,"Audi Q8 All-weather Floor Mats, Rear318",Audi Q8 Bootliner383,"Audi S8 limited, Navarra Blue, 1:43150",Audi Sport Ballpointpen197,"Audi

In [1480]:
pred_floor2_zero.drop("total",inplace=True,axis=1)

/tmp/ipykernel_31827/3942976209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_floor2_zero.drop("total",inplace=True,axis=1)


In [1481]:
pred_floor2_zero['total_floor_zero'] = pred_floor2_zero.apply(lambda row: row.sum(), axis=1)

/tmp/ipykernel_31827/3809473832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_floor2_zero['total_floor_zero'] = pred_floor2_zero.apply(lambda row: row.sum(), axis=1)


In [1482]:
pred_floor2_zero

,2-in-1 Display Cleaner278,Audi A4 Luggage Compartment Shell407,"Audi A5 Convertible, Floret Silver, 1:43427","Audi Aviator Sunglasses, Gun Metal428","Audi Baseball Cap, Black117","Audi Baseball Cap, Blue207","Audi Baseball Cap, Red121","Audi Q5, Navarra Blue, 1:43426","Audi Rs 5 Coupe, 1:43181","Audi Smart Urban Travelbag, Grey157","Audi Sport Baseball cap, Kids79","Audi Sport Mug Carbon, Grey71","Audi Sport Number plate holder, Black119","Audi USB Memory Key, 8 GB281",Battery Charger381,Business Bag373,Cockpit Care Solution406,LED gooseneck reading lamp292,Leather Care Set285,Pocket Umbrella130,"Ready-mixed Windscreen washer fluid, 3 l273",Rear Seat Bag382,Seat Back Pocket375,Size L143,Size: 158/164152,Valve Caps287,"Wheel Cleaner Gel, 500 ml276",Wheelsens288,Women's Watch72,"heritage Auto Union Streamline, Silver 1:43170",myAudi Connect80,"quattro Mug, White/black123","quattro Umbrella, Black84",55 TFSI quattro tiptronic113,55 TFSI quattro tiptronic114,Audi A6 45 TFSI Premium Plus108,Audi A6 45 TFSI Technology107,"Audi A6 Car Cover, Audi Rings399","Audi A7 Sportback, Myth Black, 1:43183",Audi Baby Mirror279,"Audi Backpack Foldable, Black201","Audi Child Seat, misano red/black402","Audi Cufflinks, Black75","Audi Q2 All-weather Floor Mats, Front301","Audi Q2 All-weather Floor Mats, Rear289","Audi Q8, Orca Black, 1:43126","Audi R8 Coupé, Camouflage Green Matt, 1:18420","Audi R8 Lms Presentation Warpaint, 1:43164",Audi RS 7 Sportback tiptronic112,"Audi Rings Decals, Floret Silver298","Audi S5 Convertible, 1:43176","Audi Smart Urban Backpack, Grey142",Audi Sport In Ear plugs60,"Audi Sport Sunglasses mirror lens, Black/red66","Audi Sport Umbrella, Black/red, Big133","Audi Umbrella, Black/silver, Big132",Espresso Mobile291,"Insect Eliminator, 500 ml274",Loading Sill Protective Mat307,Microfibre Interior Brush280,"Notebook, Din A5144",Premium plus111,Premium99,RS7 Sportback tiptronic112,RSQ8 4.0 TFSI quattro116,S5 TFSI109,Size L135,Stainless Steel Pedal Caps356,Standard with sunroof100,TEST1425,Technology104,Technology110,Tow Rope282,USB Charging Adapter (dual)297,Wireless Charging Cover for Apple iPhone 6/6S325,Wireless Charging Pad320,"heritage Sunglasses, Black/red156","quattro Ballpoint Pen With USB, 8 GB136","quattro Key Ring Carabiner, Black/silver124","Audi A4 Wind Deflector, Front364","Audi A8 L, Monsoon Grey, 1:43122","Audi Baseball Cap, White202","Audi Q7 Rubber Floor Mats, Front327","Audi Smartphone Case, Dark grey, iPhone 6/6s/7/8433","Audi Sport Mug, Red141","Audi e-tron, Antigua Blue, 1:18429",Celebration 55 TFSI quattro tiptronic115,Coat Hanger290,"Cool and Hot Box, 20 l408",Heritage wallet76,"Intensive plastic care solution, 250 ml268","Leather Care Solution, 250 ml270",Microfibre Polishing cloth267,"Plastic Cleaner, 500 ml269","Roof Luggage Box, 300 l414","Roof Luggage Box, 360 l415","Screenwash, 500 ml264",Size L175,Size L199,Size L216,Size M165,Size S184,Size S214,Size S442,Size XL160,TEST2447,"USB type-C Charging Cable Set, for Lightning & Type C devices355",e-tron Booking449,e-tron Sportback Booking451,"heritage Auto Union Typ B, Silver 1:43171","heritage Spark plug USB, 8 GB204",quattro Logo in Black339,quattro Messenger Bag203,"Active Foam Cleaner, 300 ml272",Adapter Cable Set for the Audi Music Interface477,"All-weather Floor Mats, Rear481",Anti-Marten Device393,"Audi A3 All-Weather Floor Mats, Rear491","Audi A3 Rubber Floor Mats, Rear500","Audi A6 All-weather Floor Mats, Front479",Audi A6 Bootliner380,"Audi Baby Seat, Misano red/black419",Audi Ballpoint pen205,"Audi Card Case, Black127","Audi Entertainment Mobile, Twin Package Click & Go505",Audi Q3 Bootliner494,"Audi Q3 Rubber Floor Mats, Rear466","Audi Q3 Trim Strips for Radiator Grille, Chrome Finish474",Audi Q5 Luggage Compartment Shell533,"Audi Q5 Rubber Floor Mats, Rear497",Audi Q7 Stainless Steel Pedal Caps454,"Audi Q8 All-weather Floor Mats, Rear318",Audi Q8 Bootliner383,"Audi S8 limited, Navarra Blue, 1:43150",Audi Sport Ballpointpen197,"Audi Tie, 

In [1483]:
result_ceil_0=pred_ceil2_zero["total_ceil_zero"]

In [1484]:
result_floor_0=pred_floor2_zero["total_floor_zero"]

In [1485]:
result_ceil_0

2022-11-30    42481650.0
2022-12-31    40935877.0
2023-01-31    36920334.0
2023-02-28    35512600.0
Freq: M, Name: total_ceil_zero, dtype: float64

In [1486]:
result_floor_0

2022-11-30    25046825.0
2022-12-31    24067860.0
2023-01-31    20439369.0
2023-02-28    19032021.0
Freq: M, Name: total_floor_zero, dtype: float64

# All results

## 1.Revenue predicted ceil (forecasted values +ve)

In [1487]:
result_c

total_ceil
forecast 2022-11-30  57531495.0
         2022-12-31  55539491.0
         2023-01-31  51707214.0
         2023-02-28  50298604.0

##  2.Revenue predicted floor (forecasted values +ve)

In [1488]:
result_f

total_floor
forecast 2022-11-30   57290392.0
         2022-12-31   55295954.0
         2023-01-31   51466111.0
         2023-02-28   50056818.0

## 3.Revenue predicted ceil (forecasted values zero)

In [1489]:
result_ceil_0

2022-11-30    42481650.0
2022-12-31    40935877.0
2023-01-31    36920334.0
2023-02-28    35512600.0
Freq: M, Name: total_ceil_zero, dtype: float64

##  4.Revenue predicted floor (forecasted values zero)

In [1436]:
result_floor_0


2022-11-30    25046825.0
2022-12-31    24067860.0
2023-01-31    20439369.0
2023-02-28    19032021.0
Freq: M, Name: total_floor_zero, dtype: float64

## Revenue without Prediction

In [1426]:
result_org

,total_original
month-year,
2022-11-30,49278478.0
2022-12-31,61794569.0
2023-01-31,53618304.0
2023-02-28,52069893.0


# Percentage

## 1.Ceil_diff

In [1519]:
result_c["total_ceil"]['forecast']

2022-11-30    57531495.0
2022-12-31    55539491.0
2023-01-31    51707214.0
2023-02-28    50298604.0
Freq: M, Name: total_ceil, dtype: float64

In [1521]:
ceil_difference=result_c["total_ceil"]['forecast']-result_org["total_original"]

In [1523]:
ceil_difference

2022-11-30    57531395.0
2022-12-31    55539391.0
2023-01-31    51707114.0
2023-02-28    50298504.0
Freq: M, dtype: float64

## 2.floor_diff

In [1451]:
result_f["total_floor"]['forecast']

2022-11-30    57290392.0
2022-12-31    55295954.0
2023-01-31    51466111.0
2023-02-28    50056818.0
Freq: M, Name: total_floor, dtype: float64

In [1452]:
floor_diff=result_f["total_floor"]['forecast']-result_org["total_original"]

In [1453]:
floor_diff

2022-11-30    8011914.0
2022-12-31   -6498615.0
2023-01-31   -2152193.0
2023-02-28   -2013075.0
Freq: M, dtype: float64

## 3.Ceil_diff_0

In [1492]:
ceil_diff_zero=result_ceil_0-result_org['total_original']

In [1493]:
ceil_diff_zero

2022-11-30    -6796828.0
2022-12-31   -20858692.0
2023-01-31   -16697970.0
2023-02-28   -16557293.0
Freq: M, dtype: float64

## 4.Floor_diff_0

In [1494]:
floor_diff_zero=result_floor_0-result_org['total_original']

In [1495]:
floor_diff_zero

2022-11-30   -24231653.0
2022-12-31   -37726709.0
2023-01-31   -33178935.0
2023-02-28   -33037872.0
Freq: M, dtype: float64

## per